In [1]:
from power import *
from power.systems import System3Bus
from power.systems import IEEE14
from power.systems.b6l8 import B6L8
from power.systems.ieee118 import IEEE118
import pulp as pl
import numpy as np
import json

In [25]:
def extract_and_save_results(problem: pl.LpProblem, net: object, output_filename: str = "optimization_results.json"):
    """
    Extrai as variáveis primais e duais de um problema de otimização resolvido,
    estrutura os resultados e os exporta para um arquivo JSON.

    Args:
        problem (pl.LpProblem): O objeto do problema PuLP já resolvido.
        net (object): O objeto da rede contendo as listas de 'buses' e 'lines'.
        output_filename (str): O nome do arquivo para salvar os resultados em JSON.

    Returns:
        tuple: Uma tupla contendo (primal_results, dual_results) se a solução for ótima,
               caso contrário, retorna (None, None).
    """
    # 1. Verifica o status do solver antes de prosseguir
    if problem.status != pl.LpStatusOptimal:
        print(f"\n[AVISO] A solução não é ótima ({pl.LpStatus[problem.status]}). Nenhum resultado será extraído.")
        return None, None

    # --- Extração das Variáveis Primais ---
    try:
        primal_results = {
            'geracao_pu': {gen.id: f"{gen.p_var.value():.4f}" for gen in net.generators},
            'thetas_deg': {bus.id: f"{np.rad2deg(bus.theta_var.value()):.4f}" for bus in net.buses},
            # Nota: usei 'flow_var'. Se o seu for 'flux_var', ajuste aqui.
            'fluxo_pu': {line.id: f"{line.flow_var.value():.4f}" for line in net.lines}
        }
    except AttributeError as e:
        print(f"[ERRO] Falha ao extrair variáveis primais. Verifique os nomes dos atributos (ex: p_var, theta_var): {e}")
        return None, None

    # --- Extração das Variáveis Duais (Multiplicadores de Lagrange) ---
    
    # Função auxiliar para obter o valor dual de forma segura
    def get_dual(constraint_name, multiplier=1):
        constraint = problem.constraints.get(constraint_name)
        if constraint is not None:
            return constraint.pi * multiplier
        return "N/A" # Retorna 'N/A' se a restrição não for encontrada

    dual_results = {
        'custo_marginal_de_energia_LMP': {
            # O LMP é o dual da restrição de balanço nodal (com sinal trocado)
            bus.name: f"${get_dual(f'B{bus.id}_Power_Balance'):.2f}/MWh"
            for bus in net.buses
        },
        'congestionamento_de_fluxo': {
            # Duals das restrições de limite de fluxo
            line.name: {
                # Para restrições <=, o lambda é -pi
                'limite_superior': get_dual(f'Constraint_Flow_{line.id}_Upper', multiplier=-1),
                # Para restrições >=, o lambda é pi
                'limite_inferior': get_dual(f'Constraint_Flow_{line.id}_Lower')
            }
            for line in net.lines
        },
        'limites_de_geracao': {
            gen.name: {
                'limite_superior': get_dual(f'Constraint_P{gen.id}_Upper', multiplier=-1),
                'limite_inferior': get_dual(f'Constraint_P{gen.id}_Lower')
            }
            for gen in net.generators
        }
    }
    
    # --- Montagem Final e Exportação ---
    final_results = {
        'solver_status': pl.LpStatus[problem.status],
        'custo_total': f"${problem.objective.value():.2f}",
        'primal_results': primal_results,
        'dual_results': dual_results
    }

    try:
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(final_results, f, indent=4, ensure_ascii=False, cls=NpEncoder)
        print(f"\n[SUCESSO] Resultados exportados para o arquivo '{output_filename}'")
    except Exception as e:
        print(f"\n[ERRO] Não foi possível exportar os resultados para JSON: {e}")

    return primal_results, dual_results

# Classe auxiliar para garantir que tipos NumPy sejam serializáveis em JSON
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [2]:
b3 = System3Bus()
b14 = IEEE14()
b6l8 = B6L8()
ieee118 = IEEE118()

In [51]:
class LinearDispatch3:
    def __init__(self, net: Network):
        """
        Inicializa e constrói o problema de despacho econômico linear para uma dada rede.
        """
        self.net = net
        self.problem = None

        # Initializing losses on each bus:
        for b in self.net.buses:
            b.loss = 0

        # RNG da classe
        self.rng = np.random.default_rng(seed=42)
    # ----------------------------------------------------------------OBJECTIVE FUNCTIONS------------------------------------------------------------------------------------#
    def _fob_linear_econ_dispatch(self):
        """Define a função objetivo do problema (minimizar custo total)."""
        self.problem += pl.lpSum([g.cost_b * g.p_var for g in self.net.generators]), "Min_Generation_Cost"
    
    def _fob_min_loss(self):
        self.problem += pl.lpSum([g.p_var for g in self.net.generators]), "Min_Loss"
    
    def _fob_transmission_cost(self):
        self._update_flow_sign()
        self.problem += pl.lpSum([l.flow_sign * l.flow_max_pu * l.flow_var for l in self.net.lines]), "Min_Transmission_Cost"

    # ----------------------------------------------------------------CREATE VARIABLES------------------------------------------------------------------------------------#
    def _create_theta_variable(self):
        # Ângulo das Barras
        for b in self.net.buses:
                if b.bus_type == 'Slack':
                    b.theta_var = pl.LpVariable(f"Theta{b.id}")
                    self.problem += b.theta_var <=  0, f"Constraint_Theta_{b.id}_Upper"
                    self.problem += b.theta_var >= 0, f"Constraint_Theta_{b.id}_Lower"
                    b.theta_var.setInitialValue(0)   
                else:
                    b.theta_var = pl.LpVariable(f"Theta{b.id}")
                    self.problem += b.theta_var <=  np.pi, f"Constraint_Theta_{b.id}_Upper"
                    self.problem += b.theta_var >= -np.pi, f"Constraint_Theta_{b.id}_Lower"
                    b.theta_var.setInitialValue(0)   

    def _create_flow_variable(self):
        for line in self.net.lines:
            line.flow_var = pl.LpVariable(f"Flow_{line.id}")
            self.problem += line.flow_var <=  line.flow_max_pu, f"Constraint_Flow_{line.id}_Upper"
            self.problem += line.flow_var >= -line.flow_max_pu, f"Constraint_Flow_{line.id}_Lower"
            self.problem += line.flow_var == ((line.from_bus.theta_var - line.to_bus.theta_var) / line.reactance), f"Constraint_Flow_{line.id}"
    
    def _create_generation_variable(self):
        for g in self.net.generators:
            g.p_var = pl.LpVariable(f"P{g.id}")
            self.problem += g.p_var <= g.p_max, f"Constraint_P{g.id}_Upper"
            self.problem += g.p_var >= g.p_min, f"Constraint_P{g.id}_Lower"

    # ----------------------------------------------------------------CREATE CONSTRAINTS------------------------------------------------------------------------------------#
    def _nodal_power_balance(self):
        for b in self.net.buses:
            generation = pl.lpSum([g.p_var for g in b.generators])
            load = sum([l.p for l in b.loads]) + b.loss
            flow_in = 0
            flow_out = 0
            for l in self.net.lines:
                if l.from_bus == b: #The line starts at bus 'b', so it's an outgoing flow
                    flow_out += (b.theta_var - l.to_bus.theta_var) / l.reactance
            
                elif l.to_bus == b: #The line ends at bus 'b', so it's an incoming flow
                    flow_in += (l.from_bus.theta_var - b.theta_var) / l.reactance
            self.problem += generation + flow_in - flow_out == load, f"B{b.id}_Power_Balance"

    # ----------------------------------------------------------------UTILS------------------------------------------------------------------------------------------------#

    def _update_losses(self):
        """
        Calcula as perdas com base nos ângulos da solução atual e as atualiza nas barras.
        Retorna o valor total das perdas calculadas.
        """
        current_total_loss = 0
        # 1. Zera as perdas da iteração anterior em todas as barras
        for b in self.net.buses:
            b.loss = 0

        # 2. Calcula e distribui as novas perdas
        for l in self.net.lines:
            r = l.resistance
            x = l.reactance
            # Evita divisão por zero se a linha não tiver impedância
            g_series = r / (r**2 + x**2) if (r**2 + x**2) > 0 else 0
            
            # .value() é um alias para .varValue, mais limpo de ler
            dtheta = l.from_bus.theta_var.value() - l.to_bus.theta_var.value()
            
            line_loss = g_series * (dtheta ** 2)
            current_total_loss += line_loss
            
            # Atribui metade da perda para cada barra da linha
            l.from_bus.loss += line_loss / 2
            l.to_bus.loss += line_loss / 2
            
        return current_total_loss

    def _update_flow_sign(self):
        for line in self.net.lines:
            if line.from_bus.theta_var.value() > line.to_bus.theta_var.value(): # Fluxo de "FROM" para "TO"
                line.flow_sign = 1
            elif line.from_bus.theta_var.value() < line.to_bus.theta_var.value(): #Fluxo de "TO" para "FROM"
                line.flow_sign = -1
            elif line.from_bus.theta_var.value() == line.to_bus.theta_var.value(): #Fluxo 0
                line.flow_sign = 0
            else:
                raise ValueError(f"Fluxo não foi corretamente calculado, o sentido do fluxo não é negativo, nem positivo, nem zero")
    

    
    # ----------------------------------------------------------------SOLVING----------------------------------------------------------------------------------------------#
    
    def solve_loss(self, iter_max=100, max_tol=1e-4, min_losses=False):
        """
        Resolve o despacho econômico de forma iterativa para incluir as perdas da rede.
        """
        print("Iniciando Despacho Econômico Linear com Perdas...")

        # 1. Construir o modelo base FORA do loop
        self.problem = pl.LpProblem("Linear_Economic_Dispatch", pl.LpMinimize)
        self._create_theta_variable()
        self._create_flow_variable()
        self._create_generation_variable()
        if not min_losses:
            self._fob_linear_econ_dispatch()
        else:
            self._fob_min_loss()
        self._nodal_power_balance() # Executa com perdas iniciais (zero)

        prev_total_loss = 0

        for i in range(1, iter_max + 1):
            # 2. Resolver o problema atual
            self.problem.solve()
            if self.problem.status != pl.LpStatusOptimal:
                print(f"ERRO: Solução ótima não encontrada na iteração {i}.")
                return (pl.LpStatus[self.problem.status], None, None)

            # 3. Calcular novas perdas e verificar convergência
            current_total_loss = self._update_losses()
            loss_diff = abs(current_total_loss - prev_total_loss)
            print(f"Iteração {i}: Perdas Totais = {current_total_loss:.6f} pu | Diferença = {loss_diff:.6f}")
            if loss_diff <= max_tol:
                print(f"\nConvergência atingida na iteração {i}!")
                break    
            # 4. Preparar para a próxima iteração
            prev_total_loss = current_total_loss
            
            # Remove as restrições de balanço de potência antigas
            for b in self.net.buses:
                constraint_name = f"B{b.id}_Power_Balance"
                self.problem.constraints.pop(constraint_name)
            
            # Adiciona as novas restrições de balanço com as perdas atualizadas
            self._nodal_power_balance()
        else: # Este `else` pertence ao `for`, executando se o loop terminar sem `break`
            print(f"\nAviso: Convergência não atingida após {iter_max} iterações.")

        final_cost = pl.value(self.problem.objective)
        
        return (pl.LpStatus[self.problem.status], final_cost, current_total_loss)
    
    def solve_transmission(self):
        """
        Otimização de Custo de Transmissão com injeções de potência fixas.
        Objetivo: Obter os coeficientes de Lagrange (preços-sombra) das restrições.
        """
        print("\n" + "="*80)
        print("OTIMIZAÇÃO DE CUSTO DE TRANSMISSÃO")
        print("="*80)

        #1) Resolvendo o Problema com Perdas
        print("Passo 1: Resolvendo o Despacho Econômico com Perdas para fixar a geração...")
        status, _, _ = self.solve_loss()
        
        if status != 'Optimal':
            print("[ERRO] O despacho econômico inicial falhou. Abortando a otimização de transmissão.")
            return None, None     
        
        #2) Construir o Problema de Custo de Transmissão
        self.problem = pl.LpProblem("Transmission_Cost_Optimization", pl.LpMinimize)
        self._create_theta_variable()
        self._create_flow_variable()
        # 3. Balanço nodal COM VALORES FIXOS
        for b in self.net.buses:
            generation = sum([g.p_var.value() for g in b.generators])
            demand = sum([l.p for l in b.loads]) + b.loss
            flow_out = pl.lpSum([l.flow_var for l in self.net.lines if l.from_bus == b])
            flow_in = pl.lpSum([l.flow_var for l in self.net.lines if l.to_bus == b])
            
            self.problem += flow_in - flow_out == demand - generation

        self._fob_transmission_cost()
        self.problem.solve()

        return extract_and_save_results(self.problem, self.net, "resultados_transmissao.json")



In [53]:
net = B6L8()
solver = LinearDispatch3(net)
solver.solve_transmission()


OTIMIZAÇÃO DE CUSTO DE TRANSMISSÃO
Passo 1: Resolvendo o Despacho Econômico com Perdas para fixar a geração...
Iniciando Despacho Econômico Linear com Perdas...
Iteração 1: Perdas Totais = 0.003874 pu | Diferença = 0.003874
Iteração 2: Perdas Totais = 0.003899 pu | Diferença = 0.000025

Convergência atingida na iteração 2!

[AVISO] A solução não é ótima (Infeasible). Nenhum resultado será extraído.


(None, None)

In [ ]:
net = System3Bus()
solver = LinearDispatch3(net)
status, cost, loss = solver.solve_loss(min_losses=True)

# Agora, em vez de ter toda a lógica de extração aqui, você apenas chama a função:
if status == 'Optimal':
    extract_and_save_results(solver.problem, solver.net, "meus_resultados.json")

Iniciando Despacho Econômico Linear com Perdas...
Iteração 1: Perdas Totais = 0.000415 pu | Diferença = 0.000415
Iteração 2: Perdas Totais = 0.000416 pu | Diferença = 0.000001

Convergência atingida na iteração 2!

[SUCESSO] Resultados exportados para o arquivo 'meus_resultados.json'


In [14]:
class LinearDispatch2:
    def __init__(self, net: Network):
        """
        Inicializa e constrói o problema de despacho econômico linear para uma dada rede.
        """
        self.net = net
        self.problem = None

        # Initializing losses on each bus:
        for b in self.net.buses:
            b.loss = 0

        # RNG da classe
        self.rng = np.random.default_rng(seed=42)
        
    def _create_problem(self):
        """Cria o objeto do problema PuLP."""
        self.problem = pl.LpProblem("Linear_Economic_Dispatch_OO", pl.LpMinimize)
    
    def _create_variables(self):
        """Cria as variáveis de decisão e as anexa aos objetos da rede."""
        # Geração Ativa
        for g in self.net.generators:
            g.p_var = pl.LpVariable(f"Gen_{g.name}", lowBound=g.p_min, upBound=g.p_max)

        # Ângulo das Barras
        for b in self.net.buses:
            if b.bus_type == "Slack":
                b.theta_var = pl.LpVariable(f"Angle_{b.name}", lowBound=0, upBound=0)
            else:
                b.theta_var = pl.LpVariable(f"Angle_{b.name}", lowBound=-np.pi, upBound=np.pi)
                b.theta_var.setInitialValue(0)

    def _create_objective(self):
        """Define a função objetivo do problema (minimizar custo total)."""
        self.problem += pl.lpSum([g.cost_b * g.p_var for g in self.net.generators]), "Total_Generation_Cost"

    def _create_constraints(self):
        """Cria todas as restrições do modelo."""
        # 1. Definição de Fluxo (liga as variáveis de fluxo aos ângulos)
        for line in self.net.lines:
            if line.reactance == 0:
                raise ValueError(f"Line '{line.id}' has zero reactance.")
            
            flow_eq = (line.from_bus.theta_var - line.to_bus.theta_var) / line.reactance
            self.problem += flow_eq <= line.flow_max_pu, f"Flow_limit_Max_{line.name}"
            self.problem += flow_eq >= -line.flow_max_pu, f"Flow_limit_Inverted_{line.name}"

        for b in self.net.buses:
            generation = pl.lpSum([g.p_var for g in b.generators])
            load = sum([l.p for l in b.loads]) + b.loss
            flow_in = 0
            flow_out = 0
            for l in self.net.lines:
                if l.from_bus == b: #The line starts at bus 'b', so it's an outgoing flow
                    flow_out += (b.theta_var - l.to_bus.theta_var) / l.reactance
            
                elif l.to_bus == b: #The line ends at bus 'b', so it's an incoming flow
                    flow_in += (l.from_bus.theta_var - b.theta_var) / l.reactance

            self.problem += generation + flow_in - flow_out == load, f"{b.name}_Power_Balance"
    
    def _build(self):
        """" Monta o Problema de Otimização"""
        print("Building Optimization Problem...")
        self._create_problem()
        self._create_variables()
        self._create_constraints()
        self._create_objective()

    def solve(self):
        """Resolve o problema de otimização."""
        self._build()
        print("Solving...")
        print(self.problem)
        self.problem.solve()
        return self.problem.status == pl.LpStatusOptimal

    
    def solve_with_loss(self, iter_max=10, tol=1e-10):
        if not self.solve():
            raise ValueError("Não foi possível resolver o despacho inicial sem perdas")
        
        prev_total_loss = 0
        for i in range(iter_max):
            print(f"\n--- Iniciando Iteração {i+1} de Cálculo de Perdas ---")

            for b in self.net.buses:
                b.loss = 0
            
            current_total_loss = 0
            for l in self.net.lines:
                r = l.resistance
                x = l.reactance
                g_series = r / (r**2 + x**2) if (r**2 + x**2) > 0 else 0
                dtheta = l.from_bus.theta_var.value() - l.to_bus.theta_var.value()
                line_loss = g_series * (dtheta ** 2)
                current_total_loss += line_loss
                l.from_bus.loss += line_loss / 2
                l.to_bus.loss += line_loss / 2
            
            #Verificação de Convergência
            loss_diff = abs(current_total_loss - prev_total_loss)
            print(f"Mudança nas perdas desde a última iteração: {loss_diff:.6f} pu")
            if loss_diff < tol:
                print(f"Convergência atingida. A mudança nas perdas está abaixo da tolerância")
                break
            prev_total_loss = current_total_loss

            if not self.solve():
                print(f"Aviso: Não foi possível resolver o despacho com perdas. Erro nas iteração {i}")
            
        print("\n Processo de despacho com perdas finalizado")
        
    def print_model(self):
        print(self.problem)

    def print_results(self):
            """
            Imprime um relatório completo e organizado dos resultados da última solução,
            seguindo um formato tabular e estruturado.
            """
            if self.problem is None or self.problem.status != pl.LpStatusOptimal:
                status = "Não resolvido" if self.problem is None else pl.LpStatus[self.problem.status]
                print(f"Não foi possível encontrar uma solução ótima. Status: {status}")
                return

            print("\n" + "="*90)
            print("RELATÓRIO DO DESPACHO ECONÔMICO".center(90))
            print("="*90)

            # --- 1. RESUMO GERAL DO SISTEMA ---
            total_gen = sum(g.p_var.value() for g in self.net.generators)
            total_load_demand = sum(l.p for l in self.net.loads)
            total_loss = sum(b.loss for b in self.net.buses)
            total_cost = pl.value(self.problem.objective)

            print("\n## 1. RESUMO GERAL DO SISTEMA ##")
            print(f"   - Custo Total de Operação:............ {total_cost:,.2f} $")
            print(f"   - Geração Total Despachada:........... {total_gen:.4f} p.u.")
            print(f"   - Carga Total Atendida (Demanda):..... {total_load_demand:.4f} p.u.")
            print(f"   - Perdas Totais Estimadas:............ {total_loss:.4f} p.u.")
            print(f"   - Balanço (Geração - Carga - Perdas):. {total_gen - total_load_demand - total_loss:,.6f} p.u.")
            print("-" * 90)

            # --- 2. DESPACHO DA GERAÇÃO E ANÁLISE DE LIMITES ---
            print("\n## 2. DESPACHO DA GERAÇÃO E ANÁLISE DE LIMITES ##")
            print(f"   {'Gerador':<12} {'Barra':<6} {'P (pu)':>10} {'Carga (%)':>11} {'P (MW)':>10} {'Custo($/h)':>12} {'λ Pmin':>10} {'λ Pmax':>10}")
            print(f"   {'-'*12:<12} {'-'*6:<6} {'-'*10:>10} {'-'*11:>11} {'-'*10:>10} {'-'*12:>12} {'-'*10:>10} {'-'*10:>10}")
            
            for g in self.net.generators:
                p_pu = g.p_var.value()
                p_mw = p_pu * self.net.sb # Supondo que a base de potência está em net.s_base
                cost = p_pu * g.cost_b
                
                # Calcula o carregamento do gerador em relação à sua capacidade máxima.
                loading = (p_pu / g.p_max * 100) if g.p_max > 0 else 0
                
                # Coeficientes de Lagrange (preços-sombra) para os limites Pmin e Pmax.
                reduced_cost = g.p_var.dj
                lambda_pmin = 0.0
                lambda_pmax = 0.0
                tol = 1e-6 # Tolerância para comparação de ponto flutuante

                if abs(p_pu - g.p_min) < tol:
                    lambda_pmin = reduced_cost
                elif abs(p_pu - g.p_max) < tol:
                    lambda_pmax = -reduced_cost
                    
                print(f"   {g.id:<12} {g.bus.id:<6} {p_pu:>10.4f} {loading:>10.2f}% {'':<1} {p_mw:>10.2f} {cost:>12.2f} {lambda_pmin:>10.2f} {lambda_pmax:>10.2f}")
            print("-" * 90)

            # --- 3. FLUXO DE POTÊNCIA NAS LINHAS ---
            print("\n## 3. FLUXO DE POTÊNCIA NAS LINHAS DE TRANSMISSÃO ##")
            print(f"   {'Linha (ID)':<12} {'De -> Para':<12} {'Fluxo (p.u.)':>15} {'Capacidade':>15} {'Carregamento (%)':>18}")
            print(f"   {'-'*12:<12} {'-'*12:<12} {'-'*15:>15} {'-'*15:>15} {'-'*18:>18}")
            for line in self.net.lines:
                flow = (line.from_bus.theta_var.value() - line.to_bus.theta_var.value()) / line.reactance
                capacity = line.flow_max_pu
                loading = (abs(flow) / capacity * 100) if capacity > 0 else 0
                print(f"   {line.id:<12} {str(line.from_bus.id)+' -> '+str(line.to_bus.id):<12} {flow:>15.4f} {capacity:>15.4f} {loading:>17.2f}%")
            print("-" * 90)

            # --- 4. CONDIÇÕES DAS BARRAS E CUSTOS MARGINAIS (LMP) ---
            print("\n## 4. CONDIÇÕES DAS BARRAS E CUSTOS MARGINAIS (LMP) ##")
            print(f"   {'Barra (ID)':<12} {'Nome':<15} {'Ângulo (graus)':>18} {'LMP ($/p.u.)':>18}")
            print(f"   {'-'*12:<12} {'-'*15:<15} {'-'*18:>18} {'-'*18:>18}")
            for b in self.net.buses:
                angle_deg = np.rad2deg(b.theta_var.value())
                lmp = 0.0
                
                if b.bus_type == "Slack":
                    if b.generators:
                        lmp = b.generators[0].cost_b
                    else:
                        lmp = 0.0
                else:
                    constraint_name = f"{b.name.replace(' ', '_')}_Power_Balance"
                    if constraint_name in self.problem.constraints:
                        lmp = -self.problem.constraints[constraint_name].pi
                    
                print(f"   {b.id:<12} {b.name:<15} {angle_deg:>18.2f} {lmp:>18.2f}")
            print("-" * 90)
            
            # --- 5. ANÁLISE DE CONGESTIONAMENTO ---
            print("\n## 5. ANÁLISE DE CONGESTIONAMENTO NA TRANSMISSÃO ##")
            # Cabeçalho da nova tabela, com colunas para os custos sombra superior e inferior
            print(f"   {'Linha (ID)':<15} {'C.S. Limite Superior ($/p.u.)':>35} {'C.S. Limite Inferior ($/p.u.)':>35}")
            print(f"   {'-'*15:<15} {'-'*35:>35} {'-'*35:>35}")

            # Itera sobre todas as linhas para buscar e exibir os custos sombra
            for line in self.net.lines:
                sanitized_name = line.name.replace(" ", "_")
                
                # Busca o preço dual (pi) da restrição de limite máximo (flow <= capacity)
                # O valor de 'pi' para uma restrição '<=' ativa é negativo. Multiplicamos por -1 para interpretá-lo como um custo positivo.
                pi_max = -self.problem.constraints[f"Flow_limit_Max_{sanitized_name}"].pi
                
                # Busca o preço dual (pi) da restrição de limite mínimo (flow >= -capacity)
                # O valor de 'pi' para uma restrição '>=' ativa já é positivo.
                pi_min = self.problem.constraints[f"Flow_limit_Inverted_{sanitized_name}"].pi
                
                # Imprime os valores para a linha atual. Um valor diferente de zero indica congestão.
                print(f"   {line.id:<15} {pi_max:>35.2f} {pi_min:>35.2f}")

            print("="*90)


    # --------------------------------------------------------------------------------------------------------------------------------------------------------------------#
    #Problema de Investimento em Transmissão

    def solve_transmission(self):
        """
        Otimização de Custo de Transmissão com injeções de potência fixas.
        Objetivo: Obter os coeficientes de Lagrange (preços-sombra) das restrições.
        """
        print("\n" + "="*80)
        print("OTIMIZAÇÃO DE CUSTO DE TRANSMISSÃO")
        print("="*80)

        #1) Calcular o problema de despacho com perdas
        self.solve_with_loss()

        #2) Calcular o sentido dos fluxos resultantes da otimização
        for line in self.net.lines:
            if line.from_bus.theta_var.value() > line.to_bus.theta_var.value(): # Fluxo de "FROM" para "TO"
                line.flow_sign = 1
            elif line.from_bus.theta_var.value() < line.to_bus.theta_var.value(): #Fluxo de "TO" para "FROM"
                line.flow_sign = -1
            elif line.from_bus.theta_var.value() == line.to_bus.theta_var.value(): #Fluxo 0
                line.flow_sign = 0
            else:
                raise ValueError(f"Fluxo não foi corretamente calculado, o sentido do fluxo não é negativo, nem positivo, nem zero")
        
        #3) Criar o novo problema de otimização
        self._create_problem()

        #4) Criar as variáveis do problema: Fluxo nas Linhas (variável auxiliar)
        for line in self.net.lines:
            line.flux_var = pl.LpVariable(f"Fluxo_{line.name}")

        # Ângulo das Barras
        for b in self.net.buses:
                if b.bus_type == 'Slack':
                    b.theta_var = pl.LpVariable(f"Angle_{b.name}", upBound=0, lowBound=0)
                    b.theta_var.setInitialValue(0)   
                else:
                    b.theta_var = pl.LpVariable(f"Angle_{b.name}")
                    b.theta_var.setInitialValue(0)   
        
        #5) Restrições

        #5.1) Restrição de definição de como o fluxo é calculado a partir dos thetas

        for line in self.net.lines:
            self.problem += line.flux_var <= ((line.from_bus.theta_var - line.to_bus.theta_var) / line.reactance) + 1e-4, f"Constraint_Identidade_Fluxo_Upper_{line.name}"
            self.problem += line.flux_var >= ((line.from_bus.theta_var - line.to_bus.theta_var) / line.reactance) - 1e-4, f"Constraint_Identidade_Fluxo_Lower_{line.name}"
        #5.2) Restrição de balanço de energia nodal do sistema. Agora os valores de potencia são parÂmetros, não variáveis
        for b in self.net.buses:
            #if b.bus_type != 'Slack':
            total_generation = sum(g.p_var.value() for g in b.generators)
            total_load = sum(l.p for l in self.net.loads) + b.loss
            flow_out = pl.lpSum([l.flux_var for l in self.net.lines if l.from_bus == b])
            flow_in = pl.lpSum([l.flux_var for l in self.net.lines if l.to_bus == b])
            self.problem += total_generation + flow_in - flow_out == total_load, f"Constraint_Nodal_Balance_{b.name}"
        
        #5.3) Restrição de Limite das variáveis theta
        for b in self.net.buses:
            if b.bus_type != "Slack":
                self.problem += b.theta_var <= np.pi, f"Constraint_Theta_Upper_{b.name}"
                self.problem += b.theta_var >= -np.pi, f"Constraint_Theta_Lower_{b.name}"

        #5.4) Restrições de Limite das variáveis de fluxo
        for l in self.net.lines:
            self.problem += l.flux_var <= l.flow_max_pu, f"Constraint_Flux_Upper_{l.name}"
            self.problem += l.flux_var >= -l.flow_max_pu, f"Constraint_Flux_Lower_{l.name}"
            
        
        #6) Função objetivo
        #6.1) Calcular o custo da LT:
        for line in self.net.lines:
            line.cost = line.flow_sign * line.flow_max_pu * self.rng.uniform(2, 20) #Já calcula o custo sendo do mesmo sinal do fluxo, de forma que na FOB fique tudo positivo


        self.problem += pl.lpSum([l.cost * l.flux_var for l in self.net.lines]), "Total_Generation_Cost"

        #7) Resolver o problema de otimização
        print(self.problem)
        self.problem.solve()
        print(self.problem.status)
        

        #8) Obter as variáveis primais (apenas para verificar se bateu com o problema com perdas)
        results_primal = {
            'thetas': {bus.name: f"{np.rad2deg(bus.theta_var.value())}°" for bus in self.net.buses},
            'flows' : {line.name: f"{line.flux_var.value()} MW" for line in self.net.lines}
        }

        #9) Obter os coeficientes de lagrange de todas as restrições
        results_dual = {
            'Restrições de limites de theta': {
                bus.name: {
                    # Para restrições >=, o .pi já é o lambda (custo)
                    'lambda_inferior': self.problem.constraints[f"Constraint_Theta_Lower_{bus.name}"].pi if bus.bus_type != "Slack" else 0,
                    # Para restrições <=, o lambda (custo) é - .pi
                    'lambda_superior': -self.problem.constraints[f"Constraint_Theta_Upper_{bus.name}"].pi if bus.bus_type != "Slack" else 0
                }
                for bus in self.net.buses
            },
            'Restrições de limites de Fluxo': {
                line.name: {
                    'lambda_inferior': self.problem.constraints[f"Constraint_Flux_Lower_{line.name}"].pi,
                    'lambda_superior': -self.problem.constraints[f"Constraint_Flux_Upper_{line.name}"].pi
                }
                for line in self.net.lines
            },
            'Restrições de Identidade de Fluxo': {
                line.name: {
                    'lambda_superior': self.problem.constraints[f"Constraint_Identidade_Fluxo_Upper_{line.name}"].pi,
                    'lambda_inferior': self.problem.constraints[f"Constraint_Identidade_Fluxo_Lower_{line.name}"].pi,
                }
                for line in self.net.lines
            },

            'Restrições de Balanço Nodal do Sistema': {
                b.name: {
                    'custo_marginal_lmp': -self.problem.constraints[f"Constraint_Nodal_Balance_{b.name}"].pi
                }
                for b in self.net.buses if b.bus_type != 'Slack'
            }
        }

        final_results = {
        'solver_status': pl.LpStatus[self.problem.status],
        'primal_results': results_primal,
        'dual_results': results_dual
        }
        
        # 2. Defina o nome do arquivo de saída
        output_filename = "resultados_otimizacao.json"
        
        # 3. Abra o arquivo e use json.dump() para exportar
        try:
            with open(output_filename, 'w', encoding='utf-8') as f:
                json.dump(final_results, f, indent=4, ensure_ascii=False)
            print(f"\n[SUCESSO] Resultados exportados para o arquivo '{output_filename}'")
        except Exception as e:
            print(f"\n[ERRO] Não foi possível exportar os resultados para JSON: {e}")

        # A função continua retornando os dicionários como antes
        return results_primal, results_dual


In [15]:
net = B6L8()
solver = LinearDispatch2(net)
solver.solve_with_loss()
solver.print_results()

Building Optimization Problem...
Solving...
Linear_Economic_Dispatch_OO:
MINIMIZE
1000*Gen_Generator_1 + 1000000*Gen_Generator_1001 + 1000000*Gen_Generator_1002 + 1000000*Gen_Generator_1003 + 1000000*Gen_Generator_1004 + 1000000*Gen_Generator_1005 + 1000000*Gen_Generator_1006 + 2000*Gen_Generator_2 + 3000*Gen_Generator_3 + 0.0
SUBJECT TO
Flow_limit_Max_Line_1: 10 Angle_Bus_1 - 10 Angle_Bus_2 <= 0.15

Flow_limit_Inverted_Line_1: 10 Angle_Bus_1 - 10 Angle_Bus_2 >= -0.15

Flow_limit_Max_Line_2: 5.88235294118 Angle_Bus_2 - 5.88235294118 Angle_Bus_3
 <= 0.15

Flow_limit_Inverted_Line_2: 5.88235294118 Angle_Bus_2
 - 5.88235294118 Angle_Bus_3 >= -0.15

Flow_limit_Max_Line_3: 10 Angle_Bus_3 - 10 Angle_Bus_4 <= 0.1

Flow_limit_Inverted_Line_3: 10 Angle_Bus_3 - 10 Angle_Bus_4 >= -0.1

Flow_limit_Max_Line_4: 6.66666666667 Angle_Bus_4 - 6.66666666667 Angle_Bus_5
 <= 0.25

Flow_limit_Inverted_Line_4: 6.66666666667 Angle_Bus_4
 - 6.66666666667 Angle_Bus_5 >= -0.25

Flow_limit_Max_Line_5: 5.555555555

In [59]:
net = B6L8()
solver = LinearDispatch2(net)
results_primal, results_dual = solver.solve_transmission()
print(results_primal)
print(results_dual)


OTIMIZAÇÃO DE CUSTO DE TRANSMISSÃO
Building Optimization Problem...
Solving...
Linear_Economic_Dispatch_OO:
MINIMIZE
1000*Gen_Generator_1 + 1000000*Gen_Generator_1001 + 1000000*Gen_Generator_1002 + 1000000*Gen_Generator_1003 + 1000000*Gen_Generator_1004 + 1000000*Gen_Generator_1005 + 1000000*Gen_Generator_1006 + 2000*Gen_Generator_2 + 3000*Gen_Generator_3 + 0.0
SUBJECT TO
Flow_limit_Max_Line_1: 10 Angle_Bus_1 - 10 Angle_Bus_2 <= 0.15

Flow_limit_Inverted_Line_1: 10 Angle_Bus_1 - 10 Angle_Bus_2 >= -0.15

Flow_limit_Max_Line_2: 5.88235294118 Angle_Bus_2 - 5.88235294118 Angle_Bus_3
 <= 0.15

Flow_limit_Inverted_Line_2: 5.88235294118 Angle_Bus_2
 - 5.88235294118 Angle_Bus_3 >= -0.15

Flow_limit_Max_Line_3: 10 Angle_Bus_3 - 10 Angle_Bus_4 <= 0.1

Flow_limit_Inverted_Line_3: 10 Angle_Bus_3 - 10 Angle_Bus_4 >= -0.1

Flow_limit_Max_Line_4: 6.66666666667 Angle_Bus_4 - 6.66666666667 Angle_Bus_5
 <= 0.25

Flow_limit_Inverted_Line_4: 6.66666666667 Angle_Bus_4
 - 6.66666666667 Angle_Bus_5 >= -0.25

In [ ]:
class LinearDispatch:
    def __init__(self, net: Network):
        """
        Inicializa e constrói o problema de despacho econômico linear para uma dada rede.
        """
        self.net = net
        self.problem = None

        # Initializing losses on each bus:
        for b in self.net.buses:
            b.loss = 0
        
    def _create_problem(self):
        """Cria o objeto do problema PuLP."""
        self.problem = pl.LpProblem("Linear_Economic_Dispatch_OO", pl.LpMinimize)
    
    def _create_variables(self):
        """Cria as variáveis de decisão e as anexa aos objetos da rede."""
        # Geração Ativa
        for g in self.net.generators:
            g.p_var = pl.LpVariable(f"Gen_{g.name}", lowBound=g.p_min, upBound=g.p_max)

        # Ângulo das Barras
        for b in self.net.buses:
            if b.bus_type == "Slack":
                b.theta_var = pl.LpVariable(f"Angle_{b.name}", lowBound=0, upBound=0)
            else:
                b.theta_var = pl.LpVariable(f"Angle_{b.name}", lowBound=-np.pi, upBound=np.pi)
                b.theta_var.setInitialValue(0)

    def _create_objective(self):
        """Define a função objetivo do problema (minimizar custo total)."""
        self.problem += pl.lpSum([g.cost_b * g.p_var for g in self.net.generators]), "Total_Generation_Cost"

    def _create_constraints(self):
        """Cria todas as restrições do modelo."""
        # 1. Definição de Fluxo (liga as variáveis de fluxo aos ângulos)
        for line in self.net.lines:
            if line.reactance == 0:
                raise ValueError(f"Line '{line.id}' has zero reactance.")
            
            flow_eq = (line.from_bus.theta_var - line.to_bus.theta_var) / line.reactance
            self.problem += flow_eq <= line.flow_max_pu, f"Flow_limit_Max_{line.name}"
            self.problem += flow_eq >= -line.flow_max_pu, f"Flow_limit_Inverted_{line.name}"

        for b in self.net.buses:
            generation = pl.lpSum([g.p_var for g in b.generators])
            load = sum([l.p for l in b.loads]) + b.loss
            flow_in = 0
            flow_out = 0
            for l in self.net.lines:
                if l.from_bus == b: #The line starts at bus 'b', so it's an outgoing flow
                    flow_out += (b.theta_var - l.to_bus.theta_var) / l.reactance
            
                elif l.to_bus == b: #The line ends at bus 'b', so it's an incoming flow
                    flow_in += (l.from_bus.theta_var - b.theta_var) / l.reactance

            self.problem += generation + flow_in - flow_out == load, f"{b.name}_Power_Balance"
    
    def _build(self):
        """" Monta o Problema de Otimização"""
        print("Building Optimization Problem...")
        self._create_problem()
        self._create_variables()
        self._create_constraints()
        self._create_objective()

    def solve(self):
        """Resolve o problema de otimização."""
        self._build()
        print("Solving...")
        self.problem.solve()
        return self.problem.status == pl.LpStatusOptimal

    
    def solve_with_loss(self, iter_max=10, tol=1e-10):
        if not self.solve():
            raise ValueError("Não foi possível resolver o despacho inicial sem perdas")
        
        prev_total_loss = 0
        for i in range(iter_max):
            print(f"\n--- Iniciando Iteração {i+1} de Cálculo de Perdas ---")

            for b in self.net.buses:
                b.loss = 0
            
            current_total_loss = 0
            for l in self.net.lines:
                r = l.resistance
                x = l.reactance
                g_series = r / (r**2 + x**2) if (r**2 + x**2) > 0 else 0
                dtheta = l.from_bus.theta_var.value() - l.to_bus.theta_var.value()
                line_loss = g_series * (dtheta ** 2)
                current_total_loss += line_loss
                l.from_bus.loss += line_loss / 2
                l.to_bus.loss += line_loss / 2
            
            #Verificação de Convergência
            loss_diff = abs(current_total_loss - prev_total_loss)
            print(f"Mudança nas perdas desde a última iteração: {loss_diff:.6f} pu")
            if loss_diff < tol:
                print(f"Convergência atingida. A mudança nas perdas está abaixo da tolerância")
                break
            prev_total_loss = current_total_loss

            if not self.solve():
                print(f"Aviso: Não foi possível resolver o despacho com perdas. Erro nas iteração {i}")
            
        print("\n Processo de despacho com perdas finalizado")
    
    def solve_transmission(self):
        #Etapa 1: Rodar o PL com perdas
        self.solve_with_loss()

        #Etapa 2: Armazenar as gerações ótimas
        for g in self.net.generators:
            g.p_opt = g.p_var.value()
        
        #Etapa 3: Determinar o sentido do fluxo de cada linha
        for line in self.net.lines:
            flow_value = (line.from_bus.theta_var.value() - line.to_bus.theta_var.value()) / line.reactance
            line.flow_sign = 1 if flow_value >= 0 else -1


        #Etapa 4: Montar o problema
        self._create_problem()
        
        #Etapa 5: Criar as variáveis
        # Ângulo das Barras
        for b in self.net.buses:
            if b.bus_type == "Slack":
                b.theta_var = pl.LpVariable(f"Angle_{b.name}", lowBound=0, upBound=0)
            else:
                b.theta_var = pl.LpVariable(f"Angle_{b.name}", lowBound=-np.pi, upBound=np.pi)
                b.theta_var.setInitialValue(0)
        
        #Etapa 6: Criar as restrições

        # 1. Definição de Fluxo (liga as variáveis de fluxo aos ângulos)
        for line in self.net.lines:
            if line.reactance == 0:
                raise ValueError(f"Line '{line.id}' has zero reactance.")
            
            line.flow_eq = (line.from_bus.theta_var - line.to_bus.theta_var) / line.reactance
            self.problem += line.flow_eq <= line.flow_max_pu, f"Flow_limit_Max_{line.name}"
            self.problem += line.flow_eq >= -line.flow_max_pu, f"Flow_limit_Inverted_{line.name}"

        #2. Restrição de Balanço Nodal
        for b in self.net.buses:
            generation = sum([g.p_opt for g in b.generators])
            load = sum([l.p for l in b.loads]) + b.loss
            flow_in = 0
            flow_out = 0
            for l in self.net.lines:
                if l.from_bus == b: #The line starts at bus 'b', so it's an outgoing flow
                    flow_out += (b.theta_var - l.to_bus.theta_var) / l.reactance
            
                elif l.to_bus == b: #The line ends at bus 'b', so it's an incoming flow
                    flow_in += (l.from_bus.theta_var - b.theta_var) / l.reactance

            self.problem += flow_in - flow_out == load - generation, f"{b.name}_Power_Balance"       


        #Etapa 7: Criar a FOB
        self.problem += pl.lpSum([line.flow_max_pu * line.flow_sign * line.flow_eq for line in self.net.lines]), "Total_Flow_Cost"

        #Etapa 8: Resolver o problema
        print("--> Etapa 7: Resolvendo o problema de sinais de expansão...")
        self.problem.solve()
        if self.problem.status != pl.LpStatusOptimal:
            print("\nERRO: Não foi possível resolver o problema para obter os sinais de expansão.")
            return False
        return True


    def print_model(self):
        print(self.problem)

    def print_results(self):
            """
            Imprime um relatório completo e organizado dos resultados da última solução,
            seguindo um formato tabular e estruturado.
            """
            if self.problem is None or self.problem.status != pl.LpStatusOptimal:
                status = "Não resolvido" if self.problem is None else pl.LpStatus[self.problem.status]
                print(f"Não foi possível encontrar uma solução ótima. Status: {status}")
                return

            print("\n" + "="*90)
            print("RELATÓRIO DO DESPACHO ECONÔMICO".center(90))
            print("="*90)

            # --- 1. RESUMO GERAL DO SISTEMA ---
            total_gen = sum(g.p_var.value() for g in self.net.generators)
            total_load_demand = sum(l.p for l in self.net.loads)
            total_loss = sum(b.loss for b in self.net.buses)
            total_cost = pl.value(self.problem.objective)

            print("\n## 1. RESUMO GERAL DO SISTEMA ##")
            print(f"   - Custo Total de Operação:............ {total_cost:,.2f} $")
            print(f"   - Geração Total Despachada:........... {total_gen:.4f} p.u.")
            print(f"   - Carga Total Atendida (Demanda):..... {total_load_demand:.4f} p.u.")
            print(f"   - Perdas Totais Estimadas:............ {total_loss:.4f} p.u.")
            print(f"   - Balanço (Geração - Carga - Perdas):. {total_gen - total_load_demand - total_loss:,.6f} p.u.")
            print("-" * 90)

            # --- 2. DESPACHO DA GERAÇÃO E ANÁLISE DE LIMITES ---
            print("\n## 2. DESPACHO DA GERAÇÃO E ANÁLISE DE LIMITES ##")
            print(f"   {'Gerador':<12} {'Barra':<6} {'P (pu)':>10} {'Carga (%)':>11} {'P (MW)':>10} {'Custo($/h)':>12} {'λ Pmin':>10} {'λ Pmax':>10}")
            print(f"   {'-'*12:<12} {'-'*6:<6} {'-'*10:>10} {'-'*11:>11} {'-'*10:>10} {'-'*12:>12} {'-'*10:>10} {'-'*10:>10}")
            
            for g in self.net.generators:
                p_pu = g.p_var.value()
                p_mw = p_pu * self.net.sb # Supondo que a base de potência está em net.s_base
                cost = p_pu * g.cost_b
                
                # Calcula o carregamento do gerador em relação à sua capacidade máxima.
                loading = (p_pu / g.p_max * 100) if g.p_max > 0 else 0
                
                # Coeficientes de Lagrange (preços-sombra) para os limites Pmin e Pmax.
                reduced_cost = g.p_var.dj
                lambda_pmin = 0.0
                lambda_pmax = 0.0
                tol = 1e-6 # Tolerância para comparação de ponto flutuante

                if abs(p_pu - g.p_min) < tol:
                    lambda_pmin = reduced_cost
                elif abs(p_pu - g.p_max) < tol:
                    lambda_pmax = -reduced_cost
                    
                print(f"   {g.id:<12} {g.bus.id:<6} {p_pu:>10.4f} {loading:>10.2f}% {'':<1} {p_mw:>10.2f} {cost:>12.2f} {lambda_pmin:>10.2f} {lambda_pmax:>10.2f}")
            print("-" * 90)

            # --- 3. FLUXO DE POTÊNCIA NAS LINHAS ---
            print("\n## 3. FLUXO DE POTÊNCIA NAS LINHAS DE TRANSMISSÃO ##")
            print(f"   {'Linha (ID)':<12} {'De -> Para':<12} {'Fluxo (p.u.)':>15} {'Capacidade':>15} {'Carregamento (%)':>18}")
            print(f"   {'-'*12:<12} {'-'*12:<12} {'-'*15:>15} {'-'*15:>15} {'-'*18:>18}")
            for line in self.net.lines:
                flow = (line.from_bus.theta_var.value() - line.to_bus.theta_var.value()) / line.reactance
                capacity = line.flow_max_pu
                loading = (abs(flow) / capacity * 100) if capacity > 0 else 0
                print(f"   {line.id:<12} {str(line.from_bus.id)+' -> '+str(line.to_bus.id):<12} {flow:>15.4f} {capacity:>15.4f} {loading:>17.2f}%")
            print("-" * 90)

            # --- 4. CONDIÇÕES DAS BARRAS E CUSTOS MARGINAIS (LMP) ---
            print("\n## 4. CONDIÇÕES DAS BARRAS E CUSTOS MARGINAIS (LMP) ##")
            print(f"   {'Barra (ID)':<12} {'Nome':<15} {'Ângulo (graus)':>18} {'LMP ($/p.u.)':>18}")
            print(f"   {'-'*12:<12} {'-'*15:<15} {'-'*18:>18} {'-'*18:>18}")
            for b in self.net.buses:
                angle_deg = np.rad2deg(b.theta_var.value())
                lmp = 0.0
                
                if b.bus_type == "Slack":
                    if b.generators:
                        lmp = b.generators[0].cost_b
                    else:
                        lmp = 0.0
                else:
                    constraint_name = f"{b.name.replace(' ', '_')}_Power_Balance"
                    if constraint_name in self.problem.constraints:
                        lmp = -self.problem.constraints[constraint_name].pi
                    
                print(f"   {b.id:<12} {b.name:<15} {angle_deg:>18.2f} {lmp:>18.2f}")
            print("-" * 90)
            
            # --- 5. ANÁLISE DE CONGESTIONAMENTO ---
            print("\n## 5. ANÁLISE DE CONGESTIONAMENTO NA TRANSMISSÃO ##")
            # Cabeçalho da nova tabela, com colunas para os custos sombra superior e inferior
            print(f"   {'Linha (ID)':<15} {'C.S. Limite Superior ($/p.u.)':>35} {'C.S. Limite Inferior ($/p.u.)':>35}")
            print(f"   {'-'*15:<15} {'-'*35:>35} {'-'*35:>35}")

            # Itera sobre todas as linhas para buscar e exibir os custos sombra
            for line in self.net.lines:
                sanitized_name = line.name.replace(" ", "_")
                
                # Busca o preço dual (pi) da restrição de limite máximo (flow <= capacity)
                # O valor de 'pi' para uma restrição '<=' ativa é negativo. Multiplicamos por -1 para interpretá-lo como um custo positivo.
                pi_max = -self.problem.constraints[f"Flow_limit_Max_{sanitized_name}"].pi
                
                # Busca o preço dual (pi) da restrição de limite mínimo (flow >= -capacity)
                # O valor de 'pi' para uma restrição '>=' ativa já é positivo.
                pi_min = self.problem.constraints[f"Flow_limit_Inverted_{sanitized_name}"].pi
                
                # Imprime os valores para a linha atual. Um valor diferente de zero indica congestão.
                print(f"   {line.id:<15} {pi_max:>35.2f} {pi_min:>35.2f}")

            print("="*90)

    def _generate_detailed_report(self):
        """Imprime um relatório detalhado e completo da última solução do problema."""
        if self.problem is None or self.problem.status != pl.LpStatusOptimal:
            print("Problema não resolvido ou sem solução ótima. Não é possível gerar o relatório.")
            return

        print("\n" + "="*80)
        print("RELATÓRIO DETALHADO DA ANÁLISE DE TRANSMISSÃO".center(80))
        print("="*80)

        # --- SEÇÃO 0: RESUMO GERAL ---
        fob_value = pl.value(self.problem.objective)
        total_gen = sum(g.p_opt for g in self.net.generators)
        total_load = sum(l.p for l in self.net.loads)
        total_loss = total_gen - total_load # Perdas são a diferença
        
        print("\n## 0. RESUMO GERAL DO SISTEMA ##")
        print(f"  - Valor da FOB (Custo de Fluxo Ponderado): {fob_value:,.2f} $")
        print(f"  - Geração Total (fixada):................. {total_gen:.4f} p.u.")
        print(f"  - Carga Total Atendida:................... {total_load:.4f} p.u.")
        print(f"  - Perdas Totais do Sistema:............... {total_loss:.4f} p.u.")
        
        print("-" * 80)

        # --- SEÇÃO 1: GERAÇÃO POR UNIDADE ---
        print("\n## 1. DESPACHO DA GERAÇÃO (Valores Fixados) ##")
        print(f"{'Gerador (ID)':<15} {'Barra':<10} {'Geração (p.u.)':<15}")
        for g in self.net.generators:
            print(f"{g.id:<15} {g.bus.id:<10} {g.p_opt:<15.4f}")

        print("-" * 80)

        # --- SEÇÃO 2: FLUXO NAS LINHAS ---
        print("\n## 2. FLUXO DE POTÊNCIA NAS LINHAS DE TRANSMISSÃO ##")
        print(f"{'Linha (ID)':<12} {'De -> Para':<12} {'Fluxo (p.u.)':>15} {'Capacidade':>15} {'Carregamento (%)':>18}")
        for line in self.net.lines:
            flow = line.flow_eq.value()
            capacity = line.flow_max_pu
            loading = (abs(flow) / capacity * 100) if capacity > 0 else 0
            print(f"{line.id:<12} {str(line.from_bus.id)+' -> '+str(line.to_bus.id):<12} {flow:>15.4f} {capacity:>15.4f} {loading:>17.2f}%")

        print("-" * 80)
        
        # --- SEÇÃO 3: ANÁLISE DAS RESTRIÇÕES (COEFICIENTES DE LAGRANGE) ---
        print("\n## 3. ANÁLISE ECONÔMICA DAS RESTRIÇÕES (PREÇOS-SOMBRA) ##")
        
        # 3.1 Custo Marginal de Operação (preço-sombra do balanço de potência)
        print("\n  >> Custo Marginal de Operação (CMO) por Barra")
        print(f"  {'Barra (ID)':<12} {'CMO ($/p.u.)':<15}")
        for b in self.net.buses:
            if b.bus_type != "Slack":
                constraint_name = f"{b.name.replace(' ', '_')}_Power_Balance"
                cmo = -self.problem.constraints[constraint_name].pi
                print(f"  {b.id:<12} {cmo:<15.2f}")

        # 3.2 Sinais de Congestionamento (preço-sombra dos limites de fluxo)
        print("\n  >> Sinais de Congestionamento por Linha (separados por limite)")
        print(f"  {'Linha (ID)':<12} {'Limite Superior (pi)':<25} {'Limite Inferior (pi)':<25}")
        for line in self.net.lines:
            c_max_name = f"Flow_limit_Max_{line.name.replace(' ', '_')}"
            c_min_name = f"Flow_limit_Inverted_{line.name.replace(' ', '_')}"
            
            pi_max = self.problem.constraints[c_max_name].pi
            pi_min = self.problem.constraints[c_min_name].pi
            
            # Imprime apenas se um dos limites tiver um preço-sombra relevante
            if abs(pi_max) > 1e-6 or abs(pi_min) > 1e-6:
                print(f"  {line.id:<12} {pi_max:<25.2f} {pi_min:<25.2f}")
        
        print("\n" + "="*80)

    def generate_detailed_report(self):
            """
            Imprime um relatório completo e organizado dos resultados da análise de sinais de expansão,
            seguindo um formato tabular e estruturado de 5 seções.
            Este método deve ser chamado APÓS a execução de 'solve_transmission'.
            """
            if self.problem is None or self.problem.status != pl.LpStatusOptimal:
                status = "Não resolvido" if self.problem is None else pl.LpStatus[self.problem.status]
                print(f"Não foi possível encontrar uma solução ótima para a análise de transmissão. Status: {status}")
                return

            print("\n" + "="*100)
            print("RELATÓRIO DA ANÁLISE DE SINAIS DE EXPANSÃO DA TRANSMISSÃO".center(100))
            print("="*100)

            # --- 1. RESUMO GERAL DO SISTEMA ---
            fob_value = pl.value(self.problem.objective)
            total_gen = sum(g.p_opt for g in self.net.generators)
            total_load_demand = sum(l.p for l in self.net.loads)
            total_loss = sum(b.loss for b in self.net.buses)

            print("\n## 1. RESUMO GERAL DO SISTEMA ##")
            print(f"   - Valor da FOB (Sinal de Expansão):....... {fob_value:,.2f} $")
            print(f"   - Geração Total (Fixada):................. {total_gen:.4f} p.u.")
            print(f"   - Carga Total Atendida (Demanda):......... {total_load_demand:.4f} p.u.")
            print(f"   - Perdas Totais Estimadas:................ {total_loss:.4f} p.u.")
            print(f"   - Balanço (Geração - Carga - Perdas):..... {total_gen - total_load_demand - total_loss:,.6f} p.u.")
            print("-" * 100)

            # --- 2. GERAÇÃO FIXADA (INPUT DO PROBLEMA) ---
            print("\n## 2. GERAÇÃO FIXADA PARA A ANÁLISE ##")
            print(f"   {'Gerador':<15} {'Barra':<10} {'P (pu)':>15}")
            print(f"   {'-'*15:<15} {'-'*10:<10} {'-'*15:>15}")
            for g in self.net.generators:
                print(f"   {g.id:<15} {g.bus.id:<10} {g.p_opt:>15.4f}")
            print("-" * 100)

            # --- 3. FLUXO DE POTÊNCIA NAS LINHAS ---
            print("\n## 3. FLUXO DE POTÊNCIA NAS LINHAS DE TRANSMISSÃO ##")
            print(f"   {'Linha (ID)':<12} {'De -> Para':<12} {'Fluxo (p.u.)':>15} {'Capacidade':>15} {'Carregamento (%)':>20}")
            print(f"   {'-'*12:<12} {'-'*12:<12} {'-'*15:>15} {'-'*15:>15} {'-'*20:>20}")
            for line in self.net.lines:
                # A expressão do fluxo já foi criada e está em 'line.flow_eq'
                flow = line.flow_eq.value()
                capacity = line.flow_max_pu
                loading = (abs(flow) / capacity * 100) if capacity > 0 else 0
                print(f"   {line.id:<12} {str(line.from_bus.id)+' -> '+str(line.to_bus.id):<12} {flow:>15.4f} {capacity:>15.4f} {loading:>19.2f}%")
            print("-" * 100)

            # --- 4. CONDIÇÕES DAS BARRAS E PREÇOS-SOMBRA DE ENERGIA ---
            print("\n## 4. CONDIÇÕES DAS BARRAS E PREÇOS-SOMBRA (CMO) ##")
            print(f"   {'Barra (ID)':<12} {'Nome':<15} {'Ângulo (graus)':>20} {'CMO ($/p.u.)':>20}")
            print(f"   {'-'*12:<12} {'-'*15:<15} {'-'*20:>20} {'-'*20:>20}")
            for b in self.net.buses:
                angle_deg = np.rad2deg(b.theta_var.value())
                cmo = 0.0 # Custo Marginal de Operação (ou preço-sombra da energia)
                
                # O preço-sombra vem da restrição de balanço de potência
                constraint_name = f"{b.name.replace(' ', '_')}_Power_Balance"
                if constraint_name in self.problem.constraints:
                    cmo = -self.problem.constraints[constraint_name].pi
                else:
                    # Para a barra Slack, não há restrição explícita, o preço é a referência (0 neste contexto)
                    cmo = 0.0
                    
                print(f"   {b.id:<12} {b.name:<15} {angle_deg:>20.2f} {cmo:>20.2f}")
            print("-" * 100)
            
            # --- 5. ANÁLISE DE CONGESTIONAMENTO ---
            print("\n## 5. ANÁLISE DE CONGESTIONAMENTO NA TRANSMISSÃO ##")
            # Cabeçalho da tabela, com colunas para os custos sombra superior e inferior
            print(f"   {'Linha (ID)':<15} {'C.S. Limite Superior ($/p.u.)':>35} {'C.S. Limite Inferior ($/p.u.)':>35}")
            print(f"   {'-'*15:<15} {'-'*35:>35} {'-'*35:>35}")

            # Itera sobre todas as linhas para buscar e exibir os custos sombra
            for line in self.net.lines:
                sanitized_name = line.name.replace(" ", "_")
                
                # Busca o preço dual (pi) da restrição de limite máximo (flow <= capacity)
                # O valor de 'pi' para uma restrição '<=' ativa é negativo. Multiplicamos por -1 para interpretá-lo como um custo positivo.
                pi_max = -self.problem.constraints[f"Flow_limit_Max_{sanitized_name}"].pi
                
                # Busca o preço dual (pi) da restrição de limite mínimo (flow >= -capacity)
                # O valor de 'pi' para uma restrição '>=' ativa já é positivo.
                pi_min = self.problem.constraints[f"Flow_limit_Inverted_{sanitized_name}"].pi
                
                # Imprime os valores para a linha atual. Um valor diferente de zero indica congestão.
                if pi_max > 1e-6 or pi_min > 1e-6:
                    print(f"   {line.id:<15} {pi_max:>35.8f} {pi_min:>35.8f}")

            print("="*100)

    
    def solve_transmission_2(self):
        #Etapa 1: Rodar o PL com perdas
        self.solve_with_loss()

        #Etapa 2: Armazenar as gerações ótimas
        for g in self.net.generators:
            g.p_opt = g.p_var.value()
        
        #Etapa 3: Determinar o sentido do fluxo de cada linha
        for line in self.net.lines:
            flow_value = (line.from_bus.theta_var.value() - line.to_bus.theta_var.value()) / line.reactance
            line.flow_sign = 1 if flow_value >= 0 else -1


        #Etapa 4: Montar o problema
        self._create_problem()
        
        #Etapa 5: Criar as variáveis
        # Ângulo das Barras
        for b in self.net.buses:
            if b.bus_type == "Slack":
                b.theta_var = pl.LpVariable(f"Angle_{b.name}", lowBound=0, upBound=0)
            else:
                b.theta_var = pl.LpVariable(f"Angle_{b.name}", lowBound=-np.pi, upBound=np.pi)
                b.theta_var.setInitialValue(0)
        
        # Fluxo nas linhas
        for l in self.net.lines:
            l.flux_var = pl.LpVariable(f"Fluxo na {l.name}")
            l.cost = l.flow_max_pu
        
        #Etapa 6: Criar as restrições

        # 1. Definição de Fluxo (liga as variáveis de fluxo aos ângulos)
        for line in self.net.lines:
            if line.reactance == 0:
                raise ValueError(f"Line '{line.id}' has zero reactance.")
            
            flow_from_angles = (line.from_bus.theta_var - line.to_bus.theta_var) / line.reactance
            self.problem += line.flux_var == flow_from_angles, f"Flow_Def_{line.name}" 

        #2. Restrição de Balanço Nodal
        for b in self.net.buses:
            generation = sum([g.p_opt for g in b.generators])
            load = sum([l.p for l in b.loads]) + b.loss
            flow_in = 0
            flow_out = 0
            for l in self.net.lines:
                if l.from_bus == b: #The line starts at bus 'b', so it's an outgoing flow
                    flow_out += (b.theta_var - l.to_bus.theta_var) / l.reactance
            
                elif l.to_bus == b: #The line ends at bus 'b', so it's an incoming flow
                    flow_in += (l.from_bus.theta_var - b.theta_var) / l.reactance

            self.problem += flow_in - flow_out == load - generation, f"{b.name}_Power_Balance"       


        #Etapa 7: Criar a FOB
        self.problem += pl.lpSum([line.flow_max_pu * line.flow_sign * line.flow_eq for line in self.net.lines]), "Total_Flow_Cost"

        #Etapa 8: Resolver o problema
        print("--> Etapa 7: Resolvendo o problema de sinais de expansão...")
        self.problem.solve()
        if self.problem.status != pl.LpStatusOptimal:
            print("\nERRO: Não foi possível resolver o problema para obter os sinais de expansão.")
            return False
        return True


# Resultados com Perdas

In [21]:
b6 = B6L8()
solver = LinearDispatch(b6)
solver.solve_with_loss()
solver.print_results()

Building Optimization Problem...
Solving...

--- Iniciando Iteração 1 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.003874 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 2 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000025 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 3 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000000 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 4 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000000 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 5 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000000 pu
Convergência atingida. A mudança nas perdas está abaixo da tolerância

 Processo de despacho com perdas finalizado

                             RELATÓRIO DO DESPACHO ECONÔMICO                              

## 1. RESUMO GERAL DO SISTEMA ##
   -

In [22]:
ieee118 = IEEE118()
solver = LinearDispatch(ieee118)
solver.solve_with_loss()
solver.print_results()

Building Optimization Problem...
Solving...

--- Iniciando Iteração 1 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.776661 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 2 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.013873 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 3 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000579 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 4 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000020 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 5 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000001 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 6 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000000 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 7 de Cálculo de Perdas ---

# Resultados Investimento em LT

In [20]:
ieee118 = IEEE118()
solver = LinearDispatch2(ieee118)
solver.solve_with_loss()
solver.solve_transmission2()

Building Optimization Problem...
Solving...

--- Iniciando Iteração 1 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 2.162444 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 2 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.198113 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 3 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.028316 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 4 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000004 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 5 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000004 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 6 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000002 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 7 de Cálculo de Perdas ---

Transmission_Cost_Optimization:
MINIMIZE
39669.386367499574*Angle_1 + -6211.180124223602*Angle_10 + 6878.229324543296*Angle_100 + -4452.965813900837*Angle_101 + -27723.294147712753*Angle_102 + -6005.850648707796*Angle_103 + 27729.324052853466*Angle_104 + -23123.24148620016*Angle_105 + -4628.6918334752645*Angle_106 + -9836.065573770493*Angle_107 + 51288.130235498655*Angle_108 + -46369.20384951881*Angle_109 + 43719.291576686606*Angle_11 + 18329.55621431772*Angle_110 + -23841.059602649006*Angle_111 + -12500.0*Angle_112 + -23748.424790926798*Angle_113 + 30291.603821015586*Angle_114 + -49257.75978407558*Angle_115 + -320987.6543209877*Angle_116 + -9999.999999999998*Angle_117 + -18470.25192613428*Angle_118 + -85194.39778644104*Angle_12 + -14932.014750101313*Angle_13 + 5500.308272585501*Angle_14 + 39155.09250068681*Angle_15 + 313.5747925812593*Angle_16 + -6493.45535402749*Angle_17 + -9611.792822284253*Angle_18 + -8287.581940198672*Angle_19 + 12094.887094887094*Angle_2 + 4847.331702455376*Angle

In [12]:
b6 = B6L8()
solver = LinearDispatch2(b6)
solver.solve_transmission()
solver.print_transmission_results()



          TRANSMISSION INVESTMENT ANALYSIS: STEP 1 - ECONOMIC DISPATCH          
Building Optimization Problem...
Solving...
Despacho econômico inicial resolvido. PGs ótimos obtidos.
--------------------------------------------------------------------------------

        TRANSMISSION INVESTMENT ANALYSIS: STEP 2 - MINIMUM INVESTMENT LP        
Resolvendo o problema de investimento mínimo...
Solução ótima de investimento encontrada.

                         RELATÓRIO DO INVESTIMENTO EM TRANSMISSÃO                         

## 1. RESUMO DO INVESTIMENTO ##
 - Custo Total do Investimento: 0.00 $
------------------------------------------------------------------------------------------

## 2. CAPACIDADE ADICIONADA POR LINHA (INVESTIMENTO ÓTIMO) ##
   Linha (ID)           Investimento (Δf em p.u.)     
   -------------------- ------------------------------
------------------------------------------------------------------------------------------

## 3. VARIÁVEIS DUAIS (PREÇOS SOMBRA) DAS R

In [37]:
ieee118 = IEEE118()
solver = LinearDispatch(ieee118)
solver.solve_transmission()
solver.generate_detailed_report()

Building Optimization Problem...
Solving...

--- Iniciando Iteração 1 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.776661 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 2 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.013873 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 3 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000579 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 4 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000020 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 5 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000001 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 6 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000000 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 7 de Cálculo de Perdas ---

# LT novo metodo

In [42]:
ieee118 = IEEE118()
solver = LinearDispatch(ieee118)
solver.solve_transmission_flow_model()
solver.generate_signed_flow_model_report()

--> Etapa 1: Resolvendo o Despacho Econômico com Perdas...
Building Optimization Problem...
Solving...

--- Iniciando Iteração 1 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.776661 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 2 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.013873 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 3 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000579 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 4 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000020 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 5 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000001 pu
Building Optimization Problem...
Solving...

--- Iniciando Iteração 6 de Cálculo de Perdas ---
Mudança nas perdas desde a última iteração: 0.000000 pu
Building Optimization Problem...
So